In [ ]:
# Imports

import pandas as pd
import datetime as dt
from tkinter import *
from tkcalendar import Calendar

In [ ]:
# Read Excel file

def read_excel_df():
    columns = ('Date','Description','Category','Amount')
    df = pd.read_excel('BudgetTracker.xlsx', names=columns)
    df['Date'] = pd.to_datetime(df["Date"]).dt.date
    return df

In [ ]:
# Choose date for entry

def entry_date():
    root = Tk()
    cal = Calendar(root, selectmode = 'day',
                year = 2023, month = 1,
                day = 4)
    cal.pack(pady = 20)
    def selected_date():
        print("Selected Date is: " + cal.get_date())
    def close():
        root.destroy()
    Button(root, text = "Select Date",
        command = lambda:[selected_date(), close()]).pack(pady = 20)
    root.mainloop()
    
    global entry_date_global

    entry_date_global = cal.get_date()

In [ ]:
# Transcaction description
def transaction_description():
    description = input(
        "Please enter description associated with the transaction: ")
    return description

In [ ]:
# Choose start date for filter
def start_date():
    root = Tk()
    cal = Calendar(root, selectmode = 'day',
                year = 2023, month = 1,
                day = 4)
    cal.pack(pady = 20)
    def selected_start_date():
        print("Selected Start Date is: " + cal.get_date())
    def close():
        root.destroy()
    Button(root, text = "Select Date",
        command = lambda:[selected_start_date(), close()]).pack(pady = 20)
    root.mainloop()

    global start_date_global

    start_date_global = cal.get_date()

In [ ]:
# Choose end date for filter
def end_date():
    root = Tk()
    cal = Calendar(root, selectmode = 'day',
                year = 2023, month = 1,
                day = 4)
    cal.pack(pady = 20)
    def selected_end_date():
        print("Selected End Date is: " + cal.get_date())
    def close():
        root.destroy()
    Button(root, text = "Select",
        command = lambda:[selected_end_date(), close()]).pack(pady = 20)
    root.mainloop()

    global end_date_global

    end_date_global = cal.get_date()

In [ ]:
# Choose category
def get_category():
    categories = ["Income","Expense"]

    root = Tk()

    var = StringVar(root)
    var.set(categories[0])

    dropdown = OptionMenu(root, var, *categories)
    dropdown.pack(pady = 20,padx = 20)


    def selected_category():
        print("The selected category is: " + var.get())
    
    def close():
        root.destroy()
    Button(root, text = "Select",
        command = lambda:[selected_category(), close()]).pack(pady = 20)

    root.mainloop()

    global category_global
    
    if var.get() == "Income":
        category_global = "Income"
    elif var.get() == "Expense":
        category_global = "Expense"

In [ ]:
# Choose amount
def get_amount():
    amount = input('Please enter a transaction amount: ')
    try:
        amount = float(amount)
        if amount < 0:
            raise Exception('Amount entered must be greater than zero.')
    except:
        print('Invalid entry. Please try again.')
        amount = input('Please enter a transaction amount: ')
    
    if category_global == 'Expense':
        amount = -abs(amount)
    
    return amount

In [ ]:
# Date filter
def get_filtered_df(df):
    start_date()
    start_datetime = dt.datetime.strptime(start_date_global, '%m/%d/%y').date()
    end_date()
    end_datetime = dt.datetime.strptime(end_date_global, '%m/%d/%y').date()
    df_filter = df.loc[(df['Date'] >= start_datetime) & (df['Date'] <= end_datetime)]
    return df_filter

In [ ]:
# Total income
def total_income(df):
    income = df.loc[(df['Category'] == 'Income', 'Amount')].sum()
    return income

In [ ]:
# Total expense
def total_expense(df):
    expense = df.loc[(df['Category'] == 'Expense', 'Amount')].sum()
    return expense

In [ ]:
# Main function

df = read_excel_df()
    
while True:
    row = []
        
    entry_date()
    entry_datetime = dt.datetime.strptime(entry_date_global, '%m/%d/%y').date()
    row.append(entry_datetime)

    description = transaction_description()
    row.append(description)
        
    get_category()
    row.append(category_global)
        
    amount = get_amount()
    row.append(amount)
        
    df.loc[len(df)] = row
        
    choice = input("Enter 'YES' if you would like to continue entering transactions: ")
        
    if choice.lower() != 'yes':
        writer = pd.ExcelWriter('BudgetTracker.xlsx')
        df.to_excel(writer, index=False)
        writer.save()
        print('File saved successfully.')
        break
        
df_filter = get_filtered_df(df)
print(df_filter)
        
income = total_income(df_filter)
print(f'Your total income for the filtered date range is {income}')
        
expense = abs(total_expense(df_filter))
print(f'Your total expense for the filtered date range is {expense}')
        
difference = income - expense
        
if difference > 0:
    print(f'You saved a total of {difference} for the filtered date range')
elif difference < 0:
    print(f'You lost a total of {difference} for the filtered date range')